In [0]:
# Load in one of the tables
df3 = spark.sql("select * from default.reviews_train")
print((df2.count(), len(df2.columns)))


(2904161, 17)

In [0]:
print("Before duplication removal: ", df3.count())
df2 = df3.dropDuplicates(['reviewerID', 'asin'])
print("After duplication removal: ", df2.count())

Before duplication removal: 3138710
After duplication removal: 2904161

In [0]:
df2.printSchema()
df2.show(5)

root
-- reviewID: integer (nullable = true)
-- overall: double (nullable = true)
-- verified: boolean (nullable = true)
-- reviewTime: string (nullable = true)
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- unixReviewTime: integer (nullable = true)
-- label: integer (nullable = true)

+--------+-------+--------+-----------+--------------------+----------+---------------+--------------------+----------------+--------------+-----+
reviewID|overall|verified| reviewTime| reviewerID| asin| reviewerName| reviewText| summary|unixReviewTime|label|
+--------+-------+--------+-----------+--------------------+----------+---------------+--------------------+----------------+--------------+-----+
 2740967| 5.0| true|09 12, 2014|A0001528BGUBOEVR6T5U|B002MF3CZG| igozingo| Very real| Five Stars| 1410480000| 0|
 2344349| 5.0| true|10 23, 2017|A0034986DWR7WEDQN0GV|B001ASRFCC| oliver chong| superb| Five Stars| 1508716800| 0|
 928094| 5.0| true|07 18, 2017|A00929559H7AYKDCV7UE|0008102147| Ann|great love story,...| Five Stars| 1500336000| 0|
 2422091| 5.0| true|05 19, 2016|A01299515I1M70MLGCL7|B001EAVZ9A| jason|Love these candle...| 5 stars| 1463616000| 0|
 1758493| 5.0| true|11 28, 2014|A01433701FZ0LF5G2RUL|006104055X|Kindle Customer|This book was fan...|A man to die for| 1417132800| 0|
+--------+-------+--------+-----------+--------------------+----------+---------------+--------------------+----------------+--------------+-----+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
df2 = df2.withColumn('date', from_unixtime('unixReviewTime').cast(DateType()))\
.withColumn("Year", year(from_unixtime('unixReviewTime').cast(DateType())))\
.withColumn("Month", month(from_unixtime('unixReviewTime').cast(DateType())))\
.withColumn("Quarter", quarter(from_unixtime('unixReviewTime').cast(DateType())))\
.withColumn("date",to_timestamp(col("date"))).withColumn("D_O_M", date_format(col("date"), "d"))


In [0]:
df2 = df2.withColumn("D_O_W", dayofweek(df2.date))

In [0]:
#Unreadable date format
#df3 = df2.withColumn('reviewTime',to_date(df2.reviewTime, 'yyyy-MM-dd'))

In [0]:

#from pyspark.sql.functions import *
#df3 = df2.withColumn('reviewTime',to_date(df2.reviewTime, 'yyyy-MM-dd'))\
#.withColumn('day_of_week',dayofweek(df3.reviewTime))\
#.withColumn('Month',month(df4.reviewTime))\
#.withColumn('Year',year(df5.reviewTime))\
#.withColumn('Quarter',quarter(df6.reviewTime))\
#df3.show(5)

In [0]:
df2.printSchema()

root
-- reviewID: integer (nullable = true)
-- overall: double (nullable = true)
-- verified: boolean (nullable = true)
-- reviewTime: string (nullable = true)
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- unixReviewTime: integer (nullable = true)
-- label: integer (nullable = true)
-- date: timestamp (nullable = true)
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- Quarter: integer (nullable = true)
-- D_O_M: string (nullable = true)
-- D_O_W: integer (nullable = true)

In [0]:
drop_list = ['reviewTime','reviewID','unixReviewTime']
df = df2.select([column for column in df2.columns if column not in drop_list])
df.show(5)

+-------+--------+--------------------+----------+---------------+--------------------+----------------+-----+-------------------+----+-----+-------+-----+-----+
overall|verified| reviewerID| asin| reviewerName| reviewText| summary|label| date|Year|Month|Quarter|D_O_M|D_O_W|
+-------+--------+--------------------+----------+---------------+--------------------+----------------+-----+-------------------+----+-----+-------+-----+-----+
 5.0| true|A0001528BGUBOEVR6T5U|B002MF3CZG| igozingo| Very real| Five Stars| 0|2014-09-12 00:00:00|2014| 9| 3| 12| 6|
 5.0| true|A0034986DWR7WEDQN0GV|B001ASRFCC| oliver chong| superb| Five Stars| 0|2017-10-23 00:00:00|2017| 10| 4| 23| 2|
 5.0| true|A00929559H7AYKDCV7UE|0008102147| Ann|great love story,...| Five Stars| 0|2017-07-18 00:00:00|2017| 7| 3| 18| 3|
 5.0| true|A01299515I1M70MLGCL7|B001EAVZ9A| jason|Love these candle...| 5 stars| 0|2016-05-19 00:00:00|2016| 5| 2| 19| 5|
 5.0| true|A01433701FZ0LF5G2RUL|006104055X|Kindle Customer|This book was fan...|A man to die for| 0|2014-11-28 00:00:00|2014| 11| 4| 28| 6|
+-------+--------+--------------------+----------+---------------+--------------------+----------------+-----+-------------------+----+-----+-------+-----+-----+
only showing top 5 rows

In [0]:
# Let's look at some quick summary statistics
df.describe().show()

+-------+------------------+--------------------+--------------------+------------+--------------------+--------------------+-------------------+------------------+-----------------+------------------+------------------+------------------+
summary| overall| reviewerID| asin|reviewerName| reviewText| summary| label| Year| Month| Quarter| D_O_M| D_O_W|
+-------+------------------+--------------------+--------------------+------------+--------------------+--------------------+-------------------+------------------+-----------------+------------------+------------------+------------------+
 count| 2904161| 2904161| 2904161| 2903944| 2904161| 2903810| 2904161| 2904161| 2904161| 2904161| 2904161| 2904161|
 mean| 4.317500303874337| null| 4.450394368348971E7| NaN| 8.09056868409091E7| Infinity|0.18120276389635423|2013.6055759305355|6.351432306955434| 2.452985216728687|15.699427132311191|3.9587491878032934|
 stddev|1.1238170679198427| null|2.7357530386398864E8| NaN| 8.049906653090825E8| NaN| 0.3851861541396336|3.3382656324033197|3.606895705432916|1.1531493828309634| 8.826991534472722| 1.954677397920806|
 min| 1.0|A0001528BGUBOEVR6T5U| 0000013765| | | | 0| 1996| 1| 1| 1| 1|
 max| 5.0| AZZZYAYJQSDOJ| B01HIZGKOE| ~~~|~~~~~ALL GREAT ...|~~~~(,,,___,,,)(=...| 1| 2018| 12| 4| 9| 7|
+-------+------------------+--------------------+--------------------+------------+--------------------+--------------------+-------------------+------------------+-----------------+------------------+------------------+------------------+

In [0]:
# The count of each overall rating
# 5.0 Has the most count
from pyspark.sql.functions import col
df.groupBy("overall").count().orderBy(col("overall").asc()).show()
display(df.groupBy("overall").count().orderBy("overall"))

+-------+-------+
overall| count|
+-------+-------+
 1.0| 148265|
 2.0| 128169|
 3.0| 245161|
 4.0| 514188|
 5.0|1868378|
+-------+-------+

overall,count
1.0,148266
2.0,128170
3.0,245156
4.0,514177
5.0,1868392


In [0]:
# The most common product IDs
df.groupBy("reviewerID").count().orderBy(col("count").desc()).show(10)
display(df.groupBy("reviewerID").count().orderBy(col("count").desc()).head(50))

+--------------+-----+
 reviewerID|count|
+--------------+-----+
A3V6Z4RCDGRC44| 742|
 AJKWF4W7QD4NS| 637|
A2F6N60Z96CAJI| 560|
A3W4D8XOGLWUN5| 446|
A2TCG2HV1VJP6V| 391|
A2QHS1ZCIQOL7E| 389|
A29BQ6B90Y1R5F| 367|
A1K1JW1C5CUSUZ| 329|
A2582KMXLK2P06| 317|
A119Q9NFGVOEJZ| 297|
+--------------+-----+
only showing top 10 rows

reviewerID,count
A3V6Z4RCDGRC44,742
AJKWF4W7QD4NS,637
A2F6N60Z96CAJI,560
A3W4D8XOGLWUN5,446
A2TCG2HV1VJP6V,391
A2QHS1ZCIQOL7E,389
A29BQ6B90Y1R5F,367
A1K1JW1C5CUSUZ,329
A2582KMXLK2P06,317
A119Q9NFGVOEJZ,297


In [0]:
# The most common product IDs
df.groupBy("asin").count().orderBy(col("count").desc()).show(10)
display(df.groupBy("asin").count().orderBy(col("count").desc()).head(50))

+----------+-----+
 asin|count|
+----------+-----+
000711835X|15354|
0007420412|15103|
0007548672|14316|
0007444117| 7058|
0007350899| 6599|
0007378033| 5693|
006017322X| 4655|
0007384289| 4216|
0002247399| 4119|
0007155662| 4067|
+----------+-----+
only showing top 10 rows

asin,count
000711835X,15354
0007420412,15103
0007548672,14316
0007444117,7058
0007350899,6599
0007378033,5693
006017322X,4655
0007384289,4216
0002247399,4119
0007155662,4067


In [0]:
#Class Imbalance
display(df.groupBy("label").count().orderBy("label"))

label,count
0,2377916
1,526245


In [0]:
#Check for Null 
df_Columns= ["overall","verified","reviewerID","asin","Year","Month","quarter","reviewerName","reviewText","summary","label"]
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df_Columns]).show()

+-------+--------+----------+----+----+-----+-------+------------+----------+-------+-----+
overall|verified|reviewerID|asin|Year|Month|quarter|reviewerName|reviewText|summary|label|
+-------+--------+----------+----+----+-----+-------+------------+----------+-------+-----+
 0| 0| 0| 0| 0| 0| 0| 217| 0| 351| 0|
+-------+--------+----------+----+----+-----+-------+------------+----------+-------+-----+

In [0]:
df.printSchema()

root
-- overall: double (nullable = true)
-- verified: boolean (nullable = true)
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- label: integer (nullable = true)
-- date: timestamp (nullable = true)
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- Quarter: integer (nullable = true)
-- D_O_M: string (nullable = true)
-- D_O_W: integer (nullable = true)

In [0]:
#df = df.na.drop(subset=["overall","verified","reviewerID","asin","Year","Month","quarter","reviewerName","reviewText","summary","label"])
df = df.na.drop()
df.show(5)
print((df.count(), len(df.columns)))

+-------+--------+--------------------+----------+---------------+--------------------+----------------+-----+-------------------+----+-----+-------+-----+-----+
overall|verified| reviewerID| asin| reviewerName| reviewText| summary|label| date|Year|Month|Quarter|D_O_M|D_O_W|
+-------+--------+--------------------+----------+---------------+--------------------+----------------+-----+-------------------+----+-----+-------+-----+-----+
 5.0| true|A0001528BGUBOEVR6T5U|B002MF3CZG| igozingo| Very real| Five Stars| 0|2014-09-12 00:00:00|2014| 9| 3| 12| 6|
 5.0| true|A0034986DWR7WEDQN0GV|B001ASRFCC| oliver chong| superb| Five Stars| 0|2017-10-23 00:00:00|2017| 10| 4| 23| 2|
 5.0| true|A00929559H7AYKDCV7UE|0008102147| Ann|great love story,...| Five Stars| 0|2017-07-18 00:00:00|2017| 7| 3| 18| 3|
 5.0| true|A01299515I1M70MLGCL7|B001EAVZ9A| jason|Love these candle...| 5 stars| 0|2016-05-19 00:00:00|2016| 5| 2| 19| 5|
 5.0| true|A01433701FZ0LF5G2RUL|006104055X|Kindle Customer|This book was fan...|A man to die for| 0|2014-11-28 00:00:00|2014| 11| 4| 28| 6|
+-------+--------+--------------------+----------+---------------+--------------------+----------------+-----+-------------------+----+-----+-------+-----+-----+
only showing top 5 rows

(2903593, 14)

In [0]:
from pyspark.sql.types import *
#Change Verified from Boolean to Integer
df = df.withColumn("verified", df["verified"].cast(IntegerType()))\
.withColumn("D_O_M", df["D_O_M"].cast(IntegerType()))\
.withColumn("overall", df["overall"].cast(IntegerType()))\
.withColumn("D_O_W", df["D_O_W"].cast(IntegerType()))

In [0]:
df.printSchema()
df.show(5)
print((df.count(), len(df.columns)))

root
-- overall: integer (nullable = true)
-- verified: integer (nullable = true)
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- label: integer (nullable = true)
-- date: timestamp (nullable = true)
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- Quarter: integer (nullable = true)
-- D_O_M: integer (nullable = true)
-- D_O_W: integer (nullable = true)

+-------+--------+--------------------+----------+---------------+--------------------+----------------+-----+-------------------+----+-----+-------+-----+-----+
overall|verified| reviewerID| asin| reviewerName| reviewText| summary|label| date|Year|Month|Quarter|D_O_M|D_O_W|
+-------+--------+--------------------+----------+---------------+--------------------+----------------+-----+-------------------+----+-----+-------+-----+-----+
 5| 1|A0001528BGUBOEVR6T5U|B002MF3CZG| igozingo| Very real| Five Stars| 0|2014-09-12 00:00:00|2014| 9| 3| 12| 6|
 5| 1|A0034986DWR7WEDQN0GV|B001ASRFCC| oliver chong| superb| Five Stars| 0|2017-10-23 00:00:00|2017| 10| 4| 23| 2|
 5| 1|A00929559H7AYKDCV7UE|0008102147| Ann|great love story,...| Five Stars| 0|2017-07-18 00:00:00|2017| 7| 3| 18| 3|
 5| 1|A01299515I1M70MLGCL7|B001EAVZ9A| jason|Love these candle...| 5 stars| 0|2016-05-19 00:00:00|2016| 5| 2| 19| 5|
 5| 1|A01433701FZ0LF5G2RUL|006104055X|Kindle Customer|This book was fan...|A man to die for| 0|2014-11-28 00:00:00|2014| 11| 4| 28| 6|
+-------+--------+--------------------+----------+---------------+--------------------+----------------+-----+-------------------+----+-----+-------+-----+-----+
only showing top 5 rows

(2903593, 14)

In [0]:
# set seed for reproducibility
(trainingData, testingData) = df.randomSplit([0.8, 0.2], seed = 47)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testingData.count()))

Training Dataset Count: 2323387
Test Dataset Count: 580206

In [0]:
#combine all the preprocessing steps into a pipeline.
# That way, we can run the same steps on both the training data, and testing data and beyond (new data)
# without copying and pasting any code.

from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, NGram, VectorAssembler, StringIndexer, OneHotEncoder,HashingTF, IDF, FeatureHasher 
import re
from sparknlp.annotator import ContextSpellCheckerModel
from nltk.stem import WordNetLemmatizer

# We'll tokenize the text using a simple RegexTokenizer
regexTokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W")
regexTokenizer1 = RegexTokenizer(inputCol="summary", outputCol="s_words", pattern="\\W")
regexTokenizer2 = RegexTokenizer(inputCol="reviewerName", outputCol="r_words", pattern="\\W")

#n-grams
ngram = NGram(n=2, inputCol="words", outputCol="ngrams")
ngram1 = NGram(n=2,inputCol="s_words",outputCol="s_ngrams")
ngram3 = NGram(n=3, inputCol="words", outputCol="n3grams")
ngram13 = NGram(n=3,inputCol="s_words",outputCol="s3_ngrams")
ngram4 = NGram(n=4, inputCol="words", outputCol="n4grams")
ngram14 = NGram(n=4,inputCol="s_words",outputCol="s4_ngrams")

# Remove standard Stopwords
stopwordsRemover = StopWordsRemover(inputCol="ngrams", outputCol="filtered")
stopwordsRemover1 = StopWordsRemover(inputCol="s_ngrams", outputCol="s_filtered")
stopwordsRemover3 = StopWordsRemover(inputCol="n3grams", outputCol="filtered_ng3")
stopwordsRemover13 = StopWordsRemover(inputCol="s3_ngrams", outputCol="s_filtered_ng3")
stopwordsRemover4 = StopWordsRemover(inputCol="n4grams", outputCol="filtered_ng4")
stopwordsRemover14 = StopWordsRemover(inputCol="s4_ngrams", outputCol="s_filtered_ng4")

# Vectorize the sentences using simple BOW method. Other methods are possible:
CountV = CountVectorizer(inputCol="filtered", outputCol="rawFeatures",vocabSize=10000, minDF=5)
CountV1 = CountVectorizer(inputCol="s_filtered", outputCol="s_rawFeatures", vocabSize=10000, minDF=5)
CountV2 = CountVectorizer(inputCol="r_words", outputCol="r_rawFeatures", vocabSize=10000, minDF=5)
CountV3 = CountVectorizer(inputCol="filtered_ng3", outputCol="rawFeaturesng3", minDF=5)
CountV13 = CountVectorizer(inputCol="s_filtered_ng3", outputCol="s_rawFeaturesng3", minDF=5)
CountV4 = CountVectorizer(inputCol="filtered_ng4", outputCol="rawFeaturesng4", minDF=5)
CountV14 = CountVectorizer(inputCol="s_filtered_ng4", outputCol="s_rawFeaturesng4", minDF=5)

#idf
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf1 = IDF(inputCol="s_rawFeatures", outputCol="s_features")
idf2 = IDF(inputCol="r_rawFeatures", outputCol="r_features")
idfng3 = IDF(inputCol="rawFeaturesng3", outputCol="featuresng3")
idf1ng3 = IDF(inputCol="s_rawFeaturesng3", outputCol="s_featuresng3")
idfng4 = IDF(inputCol="rawFeaturesng4", outputCol="featuresng4")
idf1ng4 = IDF(inputCol="s_rawFeaturesng4", outputCol="s_featuresng4")

#encode Overall
ndexer = StringIndexer(inputCols=["overall"], outputCols=["overall_ndexer"], handleInvalid="keep")
encoder = OneHotEncoder(inputCols=["overall_ndexer"],
                        outputCols=["overall_encode"])

ndexer1 = StringIndexer(inputCols=["asin"], outputCols=["asin_ndexer"], handleInvalid="keep")
encoder1 = OneHotEncoder(inputCols=["asin_ndexer"],
                        outputCols=["asin_encode"])

ndexer2 = StringIndexer(inputCols=["reviewerID"], outputCols=["reviewerID_ndexer"], handleInvalid="keep")
encoder2 = OneHotEncoder(inputCols=["reviewerID_ndexer"],
                        outputCols=["reviewerID_encode"])

ndexer3 = StringIndexer(inputCols=["verified"], outputCols=["verified_ndexer"], handleInvalid="keep")
encoder3 = OneHotEncoder(inputCols=["verified_ndexer"],
                        outputCols=["verified_encode"])


#Year Month 
hasher = FeatureHasher(inputCols=["Year", "Month", "Quarter","D_O_M","D_O_W"],
                       outputCol="features_date")

#Vector Assembler
vector_Assem = VectorAssembler(inputCols = ['features','s_features','r_features','overall_encode','asin_encode','reviewerID_encode','verified_encode','features_date'] , outputCol = "features3")

pipeline = Pipeline(stages=[regexTokenizer, regexTokenizer1,regexTokenizer2, ngram, ngram1,ngram3, ngram13,ngram4, ngram14, stopwordsRemover, stopwordsRemover1,stopwordsRemover3,stopwordsRemover13,stopwordsRemover4,stopwordsRemover14, CountV, CountV1, CountV2,CountV3,CountV13,CountV4,CountV14,idf, idf1, idf2,idfng3,idf1ng3,idfng4,idf1ng4, ndexer,encoder, ndexer1,encoder1, ndexer2,encoder2,ndexer3,encoder3,hasher, vector_Assem])


In [0]:
# Fit the pipeline to training documents.

pipelineFit = pipeline.fit(trainingData)
trainingDataTransformed = pipelineFit.transform(trainingData)
trainingDataTransformed.show(5)

+-------+--------+--------------+----------+---------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+-----+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------+--------------+-----------+--------------------+-----------------+--------------------+---------------+---------------+--------------------+--------------------+
overall|verified| reviewerID| asin| reviewerName| reviewText| summary|label| date|Year|Month|Quarter|D_O_M|D_O_W| words| s_words| r_words| ngrams| s_ngrams| n3grams| s3_ngrams| n4grams| s4_ngrams| filtered| s_filtered| filtered_ng3| s_filtered_ng3| filtered_ng4| s_filtered_ng4| rawFeatures| s_rawFeatures| r_rawFeatures| rawFeaturesng3| s_rawFeaturesng3| rawFeaturesng4|s_rawFeaturesng4| features| s_features| r_features| featuresng3| s_featuresng3| featuresng4|s_featuresng4|overall_ndexer|overall_encode|asin_ndexer| asin_encode|reviewerID_ndexer| reviewerID_encode|verified_ndexer|verified_encode| features_date| features3|
+-------+--------+--------------+----------+---------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+-----+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------+--------------+-----------+--------------------+-----------------+--------------------+---------------+---------------+--------------------+--------------------+
 1| 0|A100NVMFY4Z6YB|B001LOL8YG| Fatimah|It became rusty a...|I totally dislike...| 0|2016-03-25 00:00:00|2016| 3| 1| 25| 6|[it, became, rust...|[i, totally, disl...| [fatimah]|[it became, becam...|[i totally, total...|[it became rusty,...|[i totally dislik...|[it became rusty ...|[i totally dislik...|[it became, becam...|[i totally, total...|[it became rusty,...|[i totally dislik...|[it became rusty ...|[i totally dislik...|(10000,[22,34,120...|(10000,[67,326,59...| (10000,[],[])|(262144,[134,1480...|(129291,[6399,887...|(262144,[92024,14...| (63254,[],[])|(10000,[22,34,120...|(10000,[67,326,59...| (10000,[],[])|(262144,[134,1480...|(129291,[6399,887...|(262144,[92024,14...|(63254,[],[])| 3.0| (5,[3],[1.0])| 20793.0|(64274,[20793],[1...| 270371.0|(974543,[270371],...| 1.0| (2,[1],[1.0])|(262144,[38269,57...|(1330968,[22,34,1...|
 1| 0|A101ZU0K6A0I22|B0026RHI3M| Old Unc|These aren't cook...|Wellll, to start ...| 0|2014-03-02 00:00:00|2014| 3| 1| 2| 1|[these, aren, t, ...|[wellll, to, star...| [old, unc]|[these aren, aren...|[wellll to, to st...|[these aren t, ar...|[wellll to start,...|[these aren t coo...|[wellll to start ...|[these aren, aren...|[wellll to, to st...|[these aren t, ar...|[wellll to start,...|[these aren t coo...|[wellll to start ...|(10000,[67,316,43...|(10000,[1292,5878...| (10000,[259],[1.0])|(262144,[1359,147...|(129291,[8048],[1...|(262144,[122324],...| (63254,[],[])|(10000,[67,316,43...|(10000,[1292,5878...|(10000,[259],[7.2...|(262144,[1359,147...|(129291,[804

In [0]:
from pyspark.ml.classification import LogisticRegression

# More classification docs: https://spark.apache.org/docs/latest/ml-classification-regression.html

lr = LogisticRegression(maxIter=20, regParam=0.1, elasticNetParam=0, featuresCol='features3', labelCol='label')
lrModel = lr.fit(trainingDataTransformed)

In [0]:
# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = lrModel.summary

print("Training Accuracy:  " + str(trainingSummary.accuracy))
print("Training Precision: " + str(trainingSummary.precisionByLabel))
print("Training Recall:    " + str(trainingSummary.recallByLabel))
print("Training FMeasure:  " + str(trainingSummary.fMeasureByLabel()))
print("Training AUC:       " + str(trainingSummary.areaUnderROC))

Training Accuracy: 0.9157992189850421
Training Precision: [0.9228825093630918, 0.8689403126946147]
Training Recall: [0.9789844195106158, 0.6300757455121913]
Training FMeasure: [0.9501060079322957, 0.7304767848605962]
Training AUC: 0.9653453853461609

In [0]:
trainingSummary.roc.show()

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 warnings.warn(
+--------------------+--------------------+
 FPR| TPR|
+--------------------+--------------------+
 0.0| 0.0|
5.939044381480062...|0.005252849849307384|
1.045902506119055...|0.010569874787271465|
1.566225863434565...|0.015856000608474915|
1.991944973965437E-4| 0.02118490982211616|
2.549058871697195E-4| 0.02645439765737158|
3.006312731156279E-4| 0.03176904574020022|
3.521380296983754E-4| 0.03705754841653911|
4.131052109595866...| 0.04230326770044019|
4.551515428638703E-4|0.047634553769216874|
5.087606160418319E-4| 0.05291354902501402|
5.597417934757757E-4| 0.05820442855648834|
6.122997083561303E-4| 0.06348817752255635|
6.669599398316991E-4| 0.06876241906808264|
 7.17941117265643E-4| 0.07405329859955695|
7.720757695924081E-4| 0.07932991700021867|
8.162244180919059E-4| 0.08465169564845362|
 8.69307912121064E-4| 0.0899330677593862|
9.187123521085973E-4| 0.09523107785626682|
9.717958461377554E-4| 0.1005124499671994|
+--------------------+--------------------+
only showing top 20 rows

In [0]:
# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
for objective in objectiveHistory:
    print(objective)

0.47302855471804645
0.4276001761306356
0.3946841425338025
0.3598594718975645
0.3403827584230423
0.28379108473779774
0.2722666357621413
0.2722666357621413
0.2739689779357828
0.2719295545952164
0.26742665823210365
0.26346436994934547
0.26034677006754514
0.25943595323753704
0.258119149289948
0.258119149289948
0.266894285497724
0.26663510836053694
0.2657544118025803
0.26447423767032663
0.26349760783985793
0.26075248731915834
0.2600850549520441

In [0]:
testingDataTransform = pipelineFit.transform(testingData)
testingDataTransform.show(5)

+-------+--------+--------------+----------+---------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+-----+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+--------------------+-----------------+--------------------+---------------+---------------+--------------------+--------------------+
overall|verified| reviewerID| asin| reviewerName| reviewText| summary|label| date|Year|Month|Quarter|D_O_M|D_O_W| words| s_words| r_words| ngrams| s_ngrams| n3grams| s3_ngrams| n4grams| s4_ngrams| filtered| s_filtered| filtered_ng3| s_filtered_ng3| filtered_ng4| s_filtered_ng4| rawFeatures| s_rawFeatures| r_rawFeatures| rawFeaturesng3| s_rawFeaturesng3| rawFeaturesng4| s_rawFeaturesng4| features| s_features| r_features| featuresng3| s_featuresng3| featuresng4| s_featuresng4|overall_ndexer|overall_encode|asin_ndexer| asin_encode|reviewerID_ndexer| reviewerID_encode|verified_ndexer|verified_encode| features_date| features3|
+-------+--------+--------------+----------+---------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+-----+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+--------------------+-----------------+--------------------+---------------+---------------+--------------------+--------------------+
 1| 0|A104WER6UUDA2M|B001HWB68K| A. Mostajo|First of all... H...|Too much money fo...| 1|2009-09-23 00:00:00|2009| 9| 3| 23| 4|[first, of, all, ...|[too, much, money...| [a, mostajo]|[first of, of all...|[too much, much m...|[first of all, of...|[too much money, ...|[first of all hal...|[too much money f...|[first of, of all...|[too much, much m...|[first of all, of...|[too much money, ...|[first of all hal...|[too much money f...|(10000,[0,1,4,6,1...|(10000,[22,292,47...| (10000,[4],[1.0])|(262144,[3,46,84,...|(129291,[15176,17...|(262144,[10,70,18...|(63254,[14659],[1...|(10000,[0,1,4,6,1...|(10000,[22,292,47...|(10000,[4],[3.815...|(262144,[3,46,84,...|(129291,[15176,17...|(262144,[10,70,18...|(63254,[14659],[1...| 3.0| (5,[3],[1.0])| 2137.0|(64274,[2137],[1.0])| 46772.0|(974543,[46772],[...| 1.0| (2,[1],[1.0])|(262144,[38269,57...|(1330968,[0,1,4,6...|
 1| 0|A107H3XGCV6Y4Q|0060161647| Paige Turner|I have read all t...|Not realistic plo...| 0|2016-03-20 00:00:00|2016| 3| 1| 20| 1|[i, have, read, a...|[not, realistic, ...| [paige, turner]|[i have, have rea...|[not realistic, r...|[i have read, hav...|[not realistic pl...|[i have read all,...|[not realistic pl...|[i have, have rea...|[not realistic, r...|[i have read, hav...|[not realistic pl...|[i have read all,...|[not realistic pl...|(10000,[0,1,3,10,...|(10000,[1223],[1.0])|(10000,[367,923],...|(262144,[2,31,77,...| (129291,[],[])|(262144,[361,825,...| (63254,[],[])|(10000,[0,1,3,10,...|(10000,[1223],[8..

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

predictions = lrModel.transform(testingDataTransform)
#predictions.show(5)

evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
#print('Test Area Under ROC', evaluator.evaluate(predictions))

In [0]:
# Load in the tables
test_df = spark.sql("select * from default.reviews_test")
test_df.show(5)
print((test_df.count(), len(test_df.columns)))

+--------+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+
reviewID|overall|verified| reviewTime| reviewerID| asin|reviewerName| reviewText| summary|unixReviewTime|
+--------+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+
80000001| 4.0| false|07 27, 2015|A1JGAP0185YJI6|0700026657| travis|I played it a whi...|But in spite of t...| 1437955200|
80000002| 5.0| true| 03 3, 2014|A1WK5I4874S3O2|0700026657| WhiteSkull|I bought this gam...|A very good game ...| 1393804800|
80000003| 5.0| true|01 12, 2013|A1YDQQJDRHM0FJ|0001713353| Leila|I am very happy w...|One of our famili...| 1357948800|
80000004| 5.0| true|11 20, 2011|A2E6AHFDJ3JBAZ|0681795107| robosolo|I purchased two o...|Insulated stainle...| 1321747200|
80000005| 5.0| false|06 28, 2011|A38NXTZUFB1O2K|0700099867| FiSH|I'm not quite fin...| Best in the series!| 1309219200|
+--------+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+
only showing top 5 rows

(348621, 10)

In [0]:
test_df = test_df.fillna(value=' ')

In [0]:

test_df = test_df.withColumn('date', from_unixtime('unixReviewTime').cast(DateType()))\
.withColumn("Year", year(from_unixtime('unixReviewTime').cast(DateType())))\
.withColumn("Month", month(from_unixtime('unixReviewTime').cast(DateType())))\
.withColumn("Quarter", quarter(from_unixtime('unixReviewTime').cast(DateType())))\
.withColumn("date",to_timestamp(col("date"))).withColumn("D_O_M", date_format(col("date"), "d"))

In [0]:
test_df = test_df.withColumn("D_O_W", dayofweek(test_df.date))

In [0]:
test_df = test_df.withColumn("verified", test_df["verified"].cast(IntegerType()))\
.withColumn("D_O_M", test_df["D_O_M"].cast(IntegerType()))\
.withColumn("overall", test_df["overall"].cast(IntegerType()))\
.withColumn("D_O_W", test_df["D_O_W"].cast(IntegerType()))

In [0]:
drop_list = ['date']
test_df = test_df.select([column for column in test_df.columns if column not in drop_list])
test_df.show(5)
print((test_df.count(), len(test_df.columns)))

+--------+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+----+-----+-------+-----+-----+
reviewID|overall|verified| reviewTime| reviewerID| asin|reviewerName| reviewText| summary|unixReviewTime|Year|Month|Quarter|D_O_M|D_O_W|
+--------+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+----+-----+-------+-----+-----+
80000001| 4| 0|07 27, 2015|A1JGAP0185YJI6|0700026657| travis|I played it a whi...|But in spite of t...| 1437955200|2015| 7| 3| 27| 2|
80000002| 5| 1| 03 3, 2014|A1WK5I4874S3O2|0700026657| WhiteSkull|I bought this gam...|A very good game ...| 1393804800|2014| 3| 1| 3| 2|
80000003| 5| 1|01 12, 2013|A1YDQQJDRHM0FJ|0001713353| Leila|I am very happy w...|One of our famili...| 1357948800|2013| 1| 1| 12| 7|
80000004| 5| 1|11 20, 2011|A2E6AHFDJ3JBAZ|0681795107| robosolo|I purchased two o...|Insulated stainle...| 1321747200|2011| 11| 4| 20| 1|
80000005| 5| 0|06 28, 2011|A38NXTZUFB1O2K|0700099867| FiSH|I'm not quite fin...| Best in the series!| 1309219200|2011| 6| 2| 28| 3|
+--------+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+----+-----+-------+-----+-----+
only showing top 5 rows

(348621, 15)

In [0]:
test_df.printSchema()

root
-- reviewID: integer (nullable = true)
-- overall: integer (nullable = true)
-- verified: integer (nullable = true)
-- reviewTime: string (nullable = false)
-- reviewerID: string (nullable = false)
-- asin: string (nullable = false)
-- reviewerName: string (nullable = false)
-- reviewText: string (nullable = false)
-- summary: string (nullable = false)
-- unixReviewTime: integer (nullable = true)
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- Quarter: integer (nullable = true)
-- D_O_M: integer (nullable = true)
-- D_O_W: integer (nullable = true)

In [0]:
test_df_Transform = pipelineFit.transform(test_df)
test_df_Transform.show(5)

+--------+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+----+-----+-------+-----+-----+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+--------------------+-----------------+--------------------+---------------+---------------+--------------------+--------------------+
reviewID|overall|verified| reviewTime| reviewerID| asin|reviewerName| reviewText| summary|unixReviewTime|Year|Month|Quarter|D_O_M|D_O_W| words| s_words| r_words| ngrams| s_ngrams| n3grams| s3_ngrams| n4grams| s4_ngrams| filtered| s_filtered| filtered_ng3| s_filtered_ng3| filtered_ng4| s_filtered_ng4| rawFeatures| s_rawFeatures| r_rawFeatures| rawFeaturesng3| s_rawFeaturesng3| rawFeaturesng4| s_rawFeaturesng4| features| s_features| r_features| featuresng3| s_featuresng3| featuresng4| s_featuresng4|overall_ndexer|overall_encode|asin_ndexer| asin_encode|reviewerID_ndexer| reviewerID_encode|verified_ndexer|verified_encode| features_date| features3|
+--------+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+----+-----+-------+-----+-----+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+--------------------+-----------------+--------------------+---------------+---------------+--------------------+--------------------+
80000001| 4| 0|07 27, 2015|A1JGAP0185YJI6|0700026657| travis|I played it a whi...|But in spite of t...| 1437955200|2015| 7| 3| 27| 2|[i, played, it, a...|[but, in, spite, ...| [travis]|[i played, played...|[but in, in spite...|[i played it, pla...|[but in spite, in...|[i played it a, p...|[but in spite of,...|[i played, played...|[but in, in spite...|[i played it, pla...|[but in spite, in...|[i played it a, p...|[but in spite of,...|(10000,[9,10,17,2...|(10000,[45,353,37...| (10000,[449],[1.0])|(262144,[95,199,2...|(129291,[211,2216...|(262144,[623,960,...| (63254,[],[])|(10000,[9,10,17,2...|(10000,[45,353,37...|(10000,[449],[7.8...|(262144,[95,199,2...|(129291,[211,2216...|(262144,[623,960,...| (63254,[],[])| 1.0| (5,[1],[1.0])| 36760.0|(64274,[36760],[1...| 4498.0|(974543,[4498],[1...| 1.0| (2,[1],[1.0])|(262144,[38269,57...|(1330968,[9,10,17...|
80000002| 5| 1| 03 3, 2014|A1WK5I4874S3O2|0700026657| WhiteSkull|I bought this gam...|A very good game ...| 1393804800|2014| 3| 1| 3| 2|[i, bought, this,...|[a, very, good, g...|[whiteskull]|[i bought, bought...|[a very, very goo...|[i bought this, b...|[a very good, ver...|[i bought this ga...|[a very good game...|[i bought, bought...|[a very, very goo...|[i bought this, b...|[a very good, ver...|[i bought this ga...|[a very good game...|(10000,[1,3,6,10,...|(10000,[30,88,119...| (10000,[],[])|(262144,[0,12,16,...|(129291,[58,731],...|(262144,[135,144,...|(63254,[1413],[1.0])|(10000,[1,3,6,

In [0]:
predictions = lrModel.transform(test_df_Transform)

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

probelement=udf(lambda v:float(v[1]),FloatType())
submission_data = predictions.select('reviewID', probelement('probability')).withColumnRenamed('<lambda>(probability)', 'label')

In [0]:
display(submission_data.select('reviewID', 'label'))

reviewID,label
80000001,0.034335315
80000002,0.10139176
80000003,0.10573602
80000004,0.90549487
80000005,0.41153896
80000006,0.12408637
80000007,0.05657063
80000008,0.42416927
80000009,0.0851287
80000010,0.328407
